# Testing the model 

In [ ]:
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Get the data
data_df = pd.read_csv("private_dataset.txt", encoding='unicode_escape', sep="\t")

f = open("OM_D1_test_data", 'rb')
dfx = pickle.load(f)
f.close()

In [ ]:
data_df["TransactionDate"] = pd.to_datetime(data_df["TransactionDate"], format="%Y-%m-%d %H:%M:%S" )

In [ ]:
# Train data FEATURE period

train_data_feature_period_start_date = pd.to_datetime("2021-03-01 00:00:00")
train_data_feature_period_end_date = pd.to_datetime("2022-06-01 00:00:00") 

# --------------------------------------------------------

# TEST data FEATURE period

test_data_feature_period_start_date = pd.to_datetime("2021-09-01 00:00:00") 
test_data_feature_period_end_date = pd.to_datetime("2022-12-01 00:00:00")

# --------------------------------------------------------


data_df = data_df[data_df.TransactionDate < test_data_feature_period_end_date]


In [ ]:
data_df.drop(['ProductID3','Phone', 'Email', 'Email2', 'Gender', 'Payment_Info','tbr'], axis=1,inplace=True) 

In [ ]:
bask_data_omni = data_df[['CustomerID', 'TransactionDate', 'Price','Discount_Amount', 'Net_Amount', 'Quantity']]

In [ ]:
bask_data_omni = bask_data_omni.groupby(['CustomerID', 'TransactionDate']).sum()
bask_data_omni = bask_data_omni.reset_index()
sorted_df_omni = bask_data_omni.sort_values(by=['CustomerID', 'TransactionDate'],ascending=False).dropna()
sorted_df_omni.reset_index(inplace=True, drop=True)
sorted_df_omni

In [ ]:
sorted_df_omni=sorted_df_omni[sorted_df_omni.Price>0]
sorted_df_omni

In [ ]:
sorted_df_omni["discount_perc"]= (sorted_df_omni["DiscountAmount"]/sorted_df_omni["Price"])
sorted_df_omni.reset_index(drop=True,inplace=True)

In [ ]:
sorted_df_omni["days_diff_omni"] = np.nan
sorted_df_omni["is_1st_purchase_omni"] = np.nan
sorted_df_omni["delta_basket_val"] = np.nan
sorted_df_omni["delta_basket_unit"] = np.nan
sorted_df_omni["delta_discount_perc"] = np.nan
sorted_df_omni

In [ ]:
for i in range(len(sorted_df_omni.CustomerID)):
    if i != len(sorted_df_omni.CustomerID)-1:
        if sorted_df_omni.CustomerID[i] == sorted_df_omni.CustomerID[i+1]:
            sorted_df_omni["days_diff_omni"][i] = sorted_df_omni.TransactionDate[i]-sorted_df_omni.TransactionDate[i+1]
            sorted_df_omni["is_1st_purchase_omni"][i] = 0 # false


        else:
            sorted_df_omni["days_diff_omni"][i] = np.nan # 1157 days. a large number for previous purchase == Never before == First purchase
            sorted_df_omni["is_1st_purchase_omni"][i] = 1


    else:
        sorted_df_omni["days_diff_omni"][i]=np.nan
        sorted_df_omni["is_1st_purchase_omni"][i] = np.nan



In [ ]:
for i in range(len(sorted_df_omni.CustomerID)):
    if i != len(sorted_df_omni.CustomerID)-1:
        if sorted_df_omni.CustomerID[i] == sorted_df_omni.CustomerID[i+1]:
            
            sorted_df_omni["delta_basket_val"][i] = sorted_df_omni.NetAmount[i]-sorted_df_omni.NetAmount[i+1]
            sorted_df_omni["delta_basket_unit"][i] = sorted_df_omni.Cquantity[i]-sorted_df_omni.Cquantity[i+1]
            sorted_df_omni["delta_discount_perc"][i] = sorted_df_omni.discount_perc[i]-sorted_df_omni.discount_perc[i+1]

        else:
            sorted_df_omni["delta_basket_val"][i] = 0
            sorted_df_omni["delta_basket_unit"][i] = 0
            sorted_df_omni["delta_discount_perc"][i] = 0

    else:
        sorted_df_omni["delta_basket_val"][i] = 0
        sorted_df_omni["delta_basket_unit"][i] = 0
        sorted_df_omni["delta_discount_perc"][i] = 0

In [ ]:
# convert timedelta to days_diff

from datetime import datetime, timedelta
sorted_df_omni.days_diff_omni = sorted_df_omni.days_diff_omni.apply(lambda x: x.days if isinstance(x, timedelta) else x)

In [ ]:
cust_df_omni = sorted_df_omni[sorted_df_omni["is_1st_purchase_omni"]==0].groupby("CustomerID").agg(['mean',
                                                                                                    'min',
                                                                                                    'max'])

cust_df_omni.drop(["TransactionDate", "is_1st_purchase_omni"],axis=1, inplace=True)

D8_final_df = cust_df_omni.merge(dfx, how="right", on="CustomerID")

D8_final_df.head(3)

In [ ]:
D8_final_df.drop(['index', 'Category3', 'isContactable', 'City','last_coupon_type_used',
                         'isShippedToBilled_sum', 'Device', 'num_of_purchases_w_discount'],
                 axis=1,
                 inplace=True)

df = D8_final_df.copy()


df["Age"] = df.loc[:,"Age"].apply(lambda x: np.where(x<18,np.nan,x))
df["Age"] = df.loc[:,"Age"].apply(lambda x: np.where(x>100,np.nan,x))

df["purchase_freq"].replace(0,np.nan, inplace=True)

In [ ]:
# Outlier Removal

colns = df.columns

for col in colns:
    print(df[col].apply(lambda x: x > df[col].mean() + 6*df[col].std()).sum(),"outliers, 6 std far from the mean")
    print(df[col].apply(lambda x: x > df[col].mean() + 6*df[col].std()).sum() /df[col].count(),"of the column")
    print("Dropping...")
    df.drop(df[df[col].apply(lambda x: x > df[col].mean() + 6*df[col].std())==True].index, inplace=True)
    print("Dropped.")

In [ ]:
colns =["purchase_freq", "purchase_period"]

for col in colns:
    print(df[col].apply(lambda x: x < df[col].mean() - 6*df[col].std()).sum(),"outliers, 6 std far from the mean")
    print(df[col].apply(lambda x: x < df[col].mean() - 6*df[col].std()).sum() /df[col].count(),"of the column")
    print("Dropping...")
    df.drop(df[df[col].apply(lambda x: x < df[col].mean() - 6*df[col].std())==True].index, inplace=True)
    print("Dropped.")

In [ ]:
df.label.value_counts()

In [ ]:
df.drop([('Price', 'mean'), ('Price', 'min'),('Price', 'max'),'ZPL1'], axis=1, inplace=True)

# Test the Model on Test Dataset

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.pipeline import Pipeline as pipe_imb
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import make_pipeline as make_pipeline_imb

In [ ]:
X= df.drop(["CustomerID","label"],axis=1)
y= df["label"]

## Fit the selected model on whole train dataset

In [ ]:
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.pipeline import Pipeline as pipe_imb
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import make_pipeline as make_pipeline_imb

In [ ]:
import pickle 
fx = open("OM_D8_Train_Data_Clean", 'rb') 
df_train_whole = pickle.load(fx)
fx.close()

In [ ]:
X_train = df_train_whole.drop(["CustomerID","label"],axis=1)
y_train = df_train_whole["label"]

### XGB

In [ ]:
num_vars = X_train.columns
"""pipe_cat = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("one_hot", OneHotEncoder(handle_unknown="ignore"))
])
"""
pipe_num = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

ct = ColumnTransformer([
    #("categorical_vars", pipe_cat, cat_vars),
    ("numeric_vars", pipe_num, num_vars),
    
], remainder="passthrough")

In [ ]:
from sklearn.metrics import f1_score
def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

In [ ]:
# XGB

from xgboost import XGBClassifier

ovsmp_pipe = pipe_imb([('ct'        , ct),
                       ('sampler'   , RandomOverSampler(random_state=42,
                                                       sampling_strategy=0.6)),
                       ('classifier', XGBClassifier(objective='binary:logistic', 
                                                    eval_metric = f1_macro,
                                                    n_estimators=1000, 
                                                    eta=0.01,
                                                    max_depth=2, 
                                                    subsample=0.9, 
                                                    min_child_weight=100, 
                                                    gamma=5, 
                                                    reg_lambda=1, 
                                                    alpha=3, 
                                                    colsample_bytree=0.5, 

                                                   )
                                                   )
                            ]
                   )

ovsmp_pipe.fit(X_train,y_train)

## Get predictions and f1-score of test dataset

In [ ]:
y_pred = ovsmp_pipe.predict(X)
print("Test F1 score:", f1_macro(y, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,roc_auc_score,precision_score
print("Accuracy:",accuracy_score(y, y_pred))
print("AUC:",roc_auc_score(y, y_pred))
print("Precision:",precision_score(y, y_pred))
print("Recall:",recall_score(y, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y, y_pred, labels=ovsmp_pipe.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=ovsmp_pipe.classes_)
disp.plot()

plt.show()